# Data Cleaning for NOAA Solar and Geographical Event Reports
** January 2000 to December 2016 **  

** Aubrey Browne **   
** July 27, 2017 **  


In [1]:
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import re
from datetime import datetime, timedelta
%matplotlib inline

**I did have to remove all the '+' from the data before importing, which I did from the command line using:**  
perl -pi -e 's![+]!!g' *.txt 

In [2]:
# collecting all the CSVs
all_files = glob("/Users/aubreybrowne/h-alpha-time-series/full_data/*.txt")

In [4]:
## Remove top 12 lines of each csv since they are just explinations of the data. 
# for file in file_list:
#     with open(file, 'r') as fin:
#         data= fin.read().splitlines(True)
#     with open(file, 'w') as fout:
#         fout.writelines(data[12:])

In [5]:
# Putting data into a pandas dataframe
df = pd.DataFrame()
colms = ["event", 'begin', 'max', 'end', 'obs', 'q', 'type', 'loc','freq', 'particulars', 'reg_num']
for file in all_files:
    df_new = pd.read_csv(file, names=colms, delim_whitespace=True)
    df_new['date'] = int("".join(re.findall(r'\d+', file)))
    df = df.append(df_new)

In [6]:
df.head()

,event,begin,max,end,obs,q,type,loc,freq,particulars,reg_num,date
0,2310,7,0013,20,GO8,5,XRA,1-8A,C1.2,8.1E-04,NaN,20010101
1,2470,46,////,46,CUL,C,RSP,40-100,III/1,NaN,NaN,20010101
2,2320,118,0121,127,GO8,5,XRA,1-8A,C1.0,4.5E-04,NaN,20010101
3,2350,322,0334,600,LEA,G,RNS,245,68,NaN,NaN,20010101
4,2330,339,0339,339,PAL,G,RBR,245,52,NaN,NaN,20010101


In [7]:
# turning date into datetime
format = '%Y%m%d'
df.date = df['date'].apply(lambda x: datetime.strptime(str(x), format))

In [9]:
# removed all none digits from begin, end, and max columns so that they can be turned into a datetime
df['begin'].replace(regex=True,inplace=True,to_replace=r'\b\D{1}',value=r'')
df['max'].replace(regex=True,inplace=True,to_replace=r'\b\D{1}',value=r'')
df['end'].replace(regex=True,inplace=True,to_replace=r'\b\D{1}',value=r'')

In [10]:
df.head()

,event,begin,max,end,obs,q,type,loc,freq,particulars,reg_num,date
0,2310,7,0013,20,GO8,5,XRA,1-8A,C1.2,8.1E-04,NaN,2001-01-01
1,2470,46,////,46,CUL,C,RSP,40-100,III/1,NaN,NaN,2001-01-01
2,2320,118,0121,127,GO8,5,XRA,1-8A,C1.0,4.5E-04,NaN,2001-01-01
3,2350,322,0334,600,LEA,G,RNS,245,68,NaN,NaN,2001-01-01
4,2330,339,0339,339,PAL,G,RBR,245,52,NaN,NaN,2001-01-01


In [11]:
# fixing some input errors and padding the times with zeros
df['begin'] = df['begin'].apply(lambda x: str(x).zfill(4) if type(x) == int else x)

In [13]:
df['end'] = df['end'].apply(lambda x: str(x).zfill(4) if type(x) == int else x)

In [14]:
df['max'] = df['max'].apply(lambda x: str(x).zfill(4) if type(x) == int else x)

In [15]:
# Adding new column for events
df['yes_no'] = df['event'].apply(lambda x: 1 if type(x) == int else 0)

In [16]:
# Creating a new category in type column for No Event 
df['type'] = df['type'].apply(lambda x: x if type(x) == str else 'NEV')

In [17]:
df.head()

,event,begin,max,end,obs,q,type,loc,freq,particulars,reg_num,date,yes_no
0,2310,0007,0013,0020,GO8,5,XRA,1-8A,C1.2,8.1E-04,NaN,2001-01-01,1
1,2470,0046,////,0046,CUL,C,RSP,40-100,III/1,NaN,NaN,2001-01-01,1
2,2320,0118,0121,0127,GO8,5,XRA,1-8A,C1.0,4.5E-04,NaN,2001-01-01,1
3,2350,0322,0334,0600,LEA,G,RNS,245,68,NaN,NaN,2001-01-01,1
4,2330,0339,0339,0339,PAL,G,RBR,245,52,NaN,NaN,2001-01-01,1


In [18]:
df.type.unique()

array(['XRA', 'RSP', 'RNS', 'RBR', 'FLA', '3000', '204', 'EPL', 'DSF',
       '93000', 'LPS', 'SPY', 'BSL', 'NEV', 'PRO', 'XFL', '030-064', 'DSD',
       'CME', '////', 'rw', 'BSD'], dtype=object)

In [37]:
# applying datetime columns to df
time = '%H%M'
df['begin'] = df['begin'].apply(lambda x: None if x == "VENT" else x)
df['begin'] = df['begin'].apply(lambda x: datetime.strptime(x, time) if type(x) == str else x)
df['max'] = df['max'].apply(lambda x: None if x == "////" else x)
df['max'] = df['max'].apply(lambda x: None if x =='EPORTS' else x)
df['max'] = df['max'].apply(lambda x: datetime.strptime(x,time) if type(x) == str else x)

In [38]:
df.head()

,event,begin,max,end,obs,q,type,loc,freq,particulars,reg_num,date,yes_no
0,2310,1900-01-01 00:07:00,1900-01-01 00:13:00,0020,GO8,5,XRA,1-8A,C1.2,8.1E-04,NaN,2001-01-01,1
1,2470,1900-01-01 00:46:00,NaT,0046,CUL,C,RSP,40-100,III/1,NaN,NaN,2001-01-01,1
2,2320,1900-01-01 01:18:00,1900-01-01 01:21:00,0127,GO8,5,XRA,1-8A,C1.0,4.5E-04,NaN,2001-01-01,1
3,2350,1900-01-01 03:22:00,1900-01-01 03:34:00,0600,LEA,G,RNS,245,68,NaN,NaN,2001-01-01,1
4,2330,1900-01-01 03:39:00,1900-01-01 03:39:00,0339,PAL,G,RBR,245,52,NaN,NaN,2001-01-01,1


In [39]:
# finding errors with entries for the end column 
values = []
for x in df.end:
    if len(str(x)) > 4:
        values.append(x)
values

['83534502', '085634502', '091034502', '80734502', '084034502', '135572508']

In [40]:
# removing those columns since there were only six of them
df = df[~df.end.isin(values)]

In [41]:
# Finally, converting to datetime
df['end'] = df['end'].apply(lambda x: datetime.strptime(x, time) if type(x) == str else x)

In [42]:
# Creating the duration column by finding the difference between the end time and start time
df['duration'] = df['end'] - df['begin']

In [43]:
# Creating the difference between max and begin column
df['del_begin_max'] = df['max'] - df['begin']

In [44]:
# Creating the difference between max and end column 
df['del_end_max']  = df['end'] - df['max']

In [45]:
# Creating column to show if max of the event is closer to the begin time or end time 
df['closer_begin_end'] = df['del_begin_max'] > df['del_end_max']

In [46]:
# Change column from True/False to 1/0
df['closer_begin_end'] = df['closer_begin_end'].apply(lambda x: 1 if x == True else 0)

In [47]:
df.head()

,event,begin,max,end,obs,q,type,loc,freq,particulars,reg_num,date,yes_no,duration,del_begin_max,del_end_max,closer_begin_end
0,2310,1900-01-01 00:07:00,1900-01-01 00:13:00,1900-01-01 00:20:00,GO8,5,XRA,1-8A,C1.2,8.1E-04,NaN,2001-01-01,1,00:13:00,00:06:00,00:07:00,0
1,2470,1900-01-01 00:46:00,NaT,1900-01-01 00:46:00,CUL,C,RSP,40-100,III/1,NaN,NaN,2001-01-01,1,00:00:00,NaT,NaT,0
2,2320,1900-01-01 01:18:00,1900-01-01 01:21:00,1900-01-01 01:27:00,GO8,5,XRA,1-8A,C1.0,4.5E-04,NaN,2001-01-01,1,00:09:00,00:03:00,00:06:00,0
3,2350,1900-01-01 03:22:00,1900-01-01 03:34:00,1900-01-01 06:00:00,LEA,G,RNS,245,68,NaN,NaN,2001-01-01,1,02:38:00,00:12:00,02:26:00,0
4,2330,1900-01-01 03:39:00,1900-01-01 03:39:00,1900-01-01 03:39:00,PAL,G,RBR,245,52,NaN,NaN,2001-01-01,1,00:00:00,00:00:00,00:00:00,0


In [48]:
df.type.unique()

array(['XRA', 'RSP', 'RNS', 'RBR', 'FLA', '3000', '204', 'EPL', 'DSF',
       '93000', 'LPS', 'SPY', 'BSL', 'NEV', 'PRO', 'XFL', 'DSD', 'CME',
       '////', 'rw', 'BSD'], dtype=object)

In [49]:
# Fixing input errors and shifting select data one column
df[df.type == '3000'] = df.loc[df.type == '3000',['obs', 'q', 'type', 'loc', 'freq']].shift(1, axis=1)

In [50]:
# Fixing input errors and shifting select data one column 
df[df.type == '204'] = df.loc[df.type == '204',['obs', 'q', 'type', 'loc', 'freq']].shift(1, axis=1) 

In [51]:
# Fixing input errors and shifting select data one column 
df[df.type == '93000'] = df.loc[df.type == '93000',['obs', 'q', 'type', 'loc', 'freq']].shift(1, axis=1) 

In [52]:
df[df.type == 'rw']

,event,begin,max,end,obs,q,type,loc,freq,particulars,reg_num,date,yes_no,duration,del_begin_max,del_end_max,closer_begin_end
0,1670,1900-01-01 00:08:00,1900-01-01 00:09:00,1900-01-01 00:11:00,LEA,3,rw,////,SF,DSD,2280.0,2015-02-06,1.0,00:03:00,00:01:00,00:02:00,0.0


In [53]:
df.type.unique()

array(['XRA', 'RSP', 'RNS', 'RBR', 'FLA', 'EPL', 'DSF', 'LPS', 'SPY',
       'BSL', 'NEV', 'PRO', 'XFL', 'DSD', 'CME', '////', 'rw', 'BSD'], dtype=object)

In [54]:
# Removing data where type is 'rw' since the data was bad
df = df[~df.type.isin(['rw'])]

In [55]:
# Fixing input errors and shifting select data one column 
df[df.type == '////'] = df.loc[df.type == '////', ['q', 'type', 'loc', 'freq']].shift(1, axis = 1)

In [56]:
# LOOKS GOOD!
df.type.unique()

array(['XRA', 'RSP', 'RNS', 'RBR', 'FLA', 'EPL', 'DSF', 'LPS', 'SPY',
       'BSL', 'NEV', 'PRO', 'XFL', 'DSD', 'CME', 'BSD'], dtype=object)

In [58]:
# Creating dictionary for solar event type to numbers 
type_dict = {'XRA': 1, 
         'RSP': 2,
         'RNS': 3,
         'FLA': 4, 
         'EPL': 5, 
         'DSF': 6, 
         'LPS': 7, 
         'SPY': 8, 
         'BSL': 9,
         'NEV': 10, 
         'PRO': 11,
         'XFL': 12,
         'DSD': 13,
         'CME': 14,
         'BSD': 15,
         'RBR': 16
        }

In [59]:
# Replacing strings with numbers representation from above dictionary
df["type"].replace(type_dict, inplace=True)

In [60]:
# Changing duration to only minutes 
df.duration = df.duration.apply(lambda x: x.seconds/60)

In [61]:
# Filling in nulls with a zero
df.fillna(0)

,event,begin,max,end,obs,q,type,loc,freq,particulars,reg_num,date,yes_no,duration,del_begin_max,del_end_max,closer_begin_end
0,2310,1900-01-01 00:07:00,1900-01-01 00:13:00,1900-01-01 00:20:00,GO8,5,1,1-8A,C1.2,8.1E-04,0.0,2001-01-01,1.0,13.0,00:06:00,00:07:00,0.0
1,2470,1900-01-01 00:46:00,1970-01-01 00:00:00,1900-01-01 00:46:00,CUL,C,2,40-100,III/1,0,0.0,2001-01-01,1.0,0.0,00:00:00,00:00:00,0.0
2,2320,1900-01-01 01:18:00,1900-01-01 01:21:00,1900-01-01 01:27:00,GO8,5,1,1-8A,C1.0,4.5E-04,0.0,2001-01-01,1.0,9.0,00:03:00,00:06:00,0.0
3,2350,1900-01-01 03:22:00,1900-01-01 03:34:00,1900-01-01 06:00:00,LEA,G,3,245,68,0,0.0,2001-01-01,1.0,158.0,00:12:00,02:26:00,0.0
4,2330,1900-01-01 03:39:00,1900-01-01 03:39:00,1900-01-01 03:39:00,PAL,G,16,245,52,0,0.0,2001-01-01,1.0,0.0,00:00:00,00:00:00,0.0
5,2340,1900-01-01 03:47:00,1900-01-01 03:49:00,1900-01-01 03:52:00,LEA,3,4,N25E45,SF,ERU,9292.0,2001-01-01,1.0,5.0,00:02:00,00:03:00,0.0
6,2480,1900-01-01 04:52:00,1970-01-01 00:00:00,1900-01-01 04:53:00,CUL,C,2,23-180,III/1,0,0.0,2001-01-01,1.0,1.0,00:00:00,00:00:00,0.0
7,2360,1900-01-01 07:31:00,1900-01-01 07:35:00,1900-01-01 07:39:00,GO8,5,1,1-8A,C1.2,4.3E-04,9289.0,2001-01-01,1.0,8.0,00:04:00,00:04:00,0.0
8,2360,1900-01-01 07:35:00,1900-01-01 07:35:00,1900-01-01 07:38:00,LEA,3,4,S09E13,SF,9289,0.0,2001-01-01,1.0,3.0,00:00:00,00:03:00,0.0
9,2370,1900-01-01 09:33:00,1900-01-01 09:51:00,1900-01-01 10:09:00,GO8,5,1,1-8A,C1.0,2.1E-03,9283.0,2001-01-01,1.0,36.0,00:18:00,00:18:00,0.0


In [62]:
df.del_begin_max = df.del_begin_max.apply(lambda x: x.seconds/60)

In [63]:
df.del_end_max = df.del_end_max.apply(lambda x: x.seconds/60)

In [65]:
# fill in with 1 because there is was events for all null values
df.yes_no = df.yes_no.fillna(1)

In [67]:
df.yes_no.unique()

array([ 1.,  0.])

In [68]:
df['duration'].fillna(0, inplace=True)

In [69]:
df['del_begin_max'].fillna(0, inplace=True)

In [70]:
df['del_end_max'].fillna(0, inplace=True)

In [71]:
# Changing columns from datetime to integers
df.del_begin_max = df.del_begin_max.apply(lambda x: int(x))
df.duration = df.duration.apply(lambda x: int(x))

In [72]:
pd.isnull(df.duration).any()

False

In [76]:
df.fillna(0)

,event,begin,max,end,obs,q,type,loc,freq,particulars,reg_num,date,yes_no,duration,del_begin_max,del_end_max,closer_begin_end
0,2310,1900-01-01 00:07:00,1900-01-01 00:13:00,1900-01-01 00:20:00,GO8,5,1,1-8A,C1.2,8.1E-04,0.0,2001-01-01,1.0,13,6,7.0,0.0
1,2470,1900-01-01 00:46:00,1970-01-01 00:00:00,1900-01-01 00:46:00,CUL,C,2,40-100,III/1,0,0.0,2001-01-01,1.0,0,0,0.0,0.0
2,2320,1900-01-01 01:18:00,1900-01-01 01:21:00,1900-01-01 01:27:00,GO8,5,1,1-8A,C1.0,4.5E-04,0.0,2001-01-01,1.0,9,3,6.0,0.0
3,2350,1900-01-01 03:22:00,1900-01-01 03:34:00,1900-01-01 06:00:00,LEA,G,3,245,68,0,0.0,2001-01-01,1.0,158,12,146.0,0.0
4,2330,1900-01-01 03:39:00,1900-01-01 03:39:00,1900-01-01 03:39:00,PAL,G,16,245,52,0,0.0,2001-01-01,1.0,0,0,0.0,0.0
5,2340,1900-01-01 03:47:00,1900-01-01 03:49:00,1900-01-01 03:52:00,LEA,3,4,N25E45,SF,ERU,9292.0,2001-01-01,1.0,5,2,3.0,0.0
6,2480,1900-01-01 04:52:00,1970-01-01 00:00:00,1900-01-01 04:53:00,CUL,C,2,23-180,III/1,0,0.0,2001-01-01,1.0,1,0,0.0,0.0
7,2360,1900-01-01 07:31:00,1900-01-01 07:35:00,1900-01-01 07:39:00,GO8,5,1,1-8A,C1.2,4.3E-04,9289.0,2001-01-01,1.0,8,4,4.0,0.0
8,2360,1900-01-01 07:35:00,1900-01-01 07:35:00,1900-01-01 07:38:00,LEA,3,4,S09E13,SF,9289,0.0,2001-01-01,1.0,3,0,3.0,0.0
9,2370,1900-01-01 09:33:00,1900-01-01 09:51:00,1900-01-01 10:09:00,GO8,5,1,1-8A,C1.0,2.1E-03,9283.0,2001-01-01,1.0,36,18,18.0,0.0


In [77]:
df.closer_begin_end.unique()

array([  0.,   1.,  nan])

In [78]:
# Finding errors 
for x in df.duration:
    if x == 'NaN':
        print(x)

In [79]:
df.duration = df.duration.apply(lambda x: int(x))

In [80]:
df.isnull().any()

event                True
begin                True
max                  True
end                  True
obs                  True
q                    True
type                False
loc                  True
freq                 True
particulars          True
reg_num              True
date                 True
yes_no              False
duration            False
del_begin_max       False
del_end_max         False
closer_begin_end     True
dtype: bool

In [81]:
# A lot of missing information in these rows so I decided to remove them
df = df[~df.date.isnull()]

In [82]:
df.isnull().any()

event               False
begin                True
max                  True
end                  True
obs                  True
q                    True
type                False
loc                  True
freq                 True
particulars          True
reg_num              True
date                False
yes_no              False
duration            False
del_begin_max       False
del_end_max         False
closer_begin_end    False
dtype: bool

In [83]:
# Replacing datetime to just time
df.begin = df.begin.apply(lambda x: x.time() if type(x) == pd._libs.tslib.Timestamp else x)

In [84]:
df.head(1)

,event,begin,max,end,obs,q,type,loc,freq,particulars,reg_num,date,yes_no,duration,del_begin_max,del_end_max,closer_begin_end
0,2310,00:07:00,1900-01-01 00:13:00,1900-01-01 00:20:00,GO8,5,1,1-8A,C1.2,8.1E-04,NaN,2001-01-01,1.0,13,6,7.0,0.0


In [86]:
df['begin'] = df['begin'].apply(lambda x: pd.to_timedelta(x) if x == pd._libs.tslib.Timestamp else x)

In [87]:
# Creating flare event column 
df['flare_event'] = df['type'].apply(lambda x: 1 if x == 4 else 0) 

In [88]:
df.head(1)

,event,begin,max,end,obs,q,type,loc,freq,particulars,reg_num,date,yes_no,duration,del_begin_max,del_end_max,closer_begin_end,flare_event
0,2310,00:07:00,1900-01-01 00:13:00,1900-01-01 00:20:00,GO8,5,1,1-8A,C1.2,8.1E-04,NaN,2001-01-01,1.0,13,6,7.0,0.0,0


In [90]:
df.t_since_last_event.unique()

array([0], dtype='timedelta64[ns]')

In [91]:
flare = df[df.flare_event == 1]

In [92]:
grouped_sum = flare.groupby(['date']).sum()
grouped_sum.head()

,type,reg_num,yes_no,duration,del_begin_max,del_end_max,closer_begin_end,flare_event
date,,,,,,,,
2001-01-01,16,18575.0,4.0,55,12,43.0,0.0,4
2001-01-03,12,27873.0,3.0,30,5,25.0,0.0,3
2001-01-04,12,9289.0,3.0,15,1,14.0,0.0,3
2001-01-05,16,27906.0,4.0,77,21,56.0,1.0,4
2001-01-06,12,NaN,3.0,76,24,52.0,0.0,3


In [93]:
count = 0
bad = 0
for x in df.end:
    if type(x) == pd._libs.tslib.Timestamp:
        count += 1
    else:
        bad += 1

In [94]:
count, bad

(123566, 987)

In [95]:
# Replacing null values for begin column 
df['begin'] = df['begin'].apply(lambda x: datetime.strptime('0000', time).time() if type(x) == pd._libs.tslib.NaTType else x)   

In [96]:
# Creating complete datetime column 
df['datetime'] = df.apply(lambda x: datetime.combine(x['date'], x['begin']), axis=1)

In [97]:
# Creating columns to show time since last event (would have been better to do this with hours and minutes)
mask = df['flare_event'].notnull()
df['last_event_time'] = np.NaN
df.loc[mask, 'last_event_time'] = df.loc[mask, 'datetime']
df['last_event_time'] = df['last_event_time'].fillna(method='ffill')
df['t_since_last_event'] = df['datetime'] - df['last_event_time']

In [98]:
df.head()

,event,begin,max,end,obs,q,type,loc,freq,particulars,...,date,yes_no,duration,del_begin_max,del_end_max,closer_begin_end,flare_event,last_event_time,t_since_last_event,datetime
0,2310,00:07:00,1900-01-01 00:13:00,1900-01-01 00:20:00,GO8,5,1,1-8A,C1.2,8.1E-04,...,2001-01-01,1.0,13,6,7.0,0.0,0,2001-01-01 00:07:00,0 days,2001-01-01 00:07:00
1,2470,00:46:00,NaT,1900-01-01 00:46:00,CUL,C,2,40-100,III/1,NaN,...,2001-01-01,1.0,0,0,0.0,0.0,0,2001-01-01 00:46:00,0 days,2001-01-01 00:46:00
2,2320,01:18:00,1900-01-01 01:21:00,1900-01-01 01:27:00,GO8,5,1,1-8A,C1.0,4.5E-04,...,2001-01-01,1.0,9,3,6.0,0.0,0,2001-01-01 01:18:00,0 days,2001-01-01 01:18:00
3,2350,03:22:00,1900-01-01 03:34:00,1900-01-01 06:00:00,LEA,G,3,245,68,NaN,...,2001-01-01,1.0,158,12,146.0,0.0,0,2001-01-01 03:22:00,0 days,2001-01-01 03:22:00
4,2330,03:39:00,1900-01-01 03:39:00,1900-01-01 03:39:00,PAL,G,16,245,52,NaN,...,2001-01-01,1.0,0,0,0.0,0.0,0,2001-01-01 03:39:00,0 days,2001-01-01 03:39:00


In [99]:
# Changing to timestamp
df['datetime'] = df.datetime.apply(lambda x: (x).timestamp())

In [100]:
df.head()

,event,begin,max,end,obs,q,type,loc,freq,particulars,...,date,yes_no,duration,del_begin_max,del_end_max,closer_begin_end,flare_event,last_event_time,t_since_last_event,datetime
0,2310,00:07:00,1900-01-01 00:13:00,1900-01-01 00:20:00,GO8,5,1,1-8A,C1.2,8.1E-04,...,2001-01-01,1.0,13,6,7.0,0.0,0,2001-01-01 00:07:00,0 days,978336420.0
1,2470,00:46:00,NaT,1900-01-01 00:46:00,CUL,C,2,40-100,III/1,NaN,...,2001-01-01,1.0,0,0,0.0,0.0,0,2001-01-01 00:46:00,0 days,978338760.0
2,2320,01:18:00,1900-01-01 01:21:00,1900-01-01 01:27:00,GO8,5,1,1-8A,C1.0,4.5E-04,...,2001-01-01,1.0,9,3,6.0,0.0,0,2001-01-01 01:18:00,0 days,978340680.0
3,2350,03:22:00,1900-01-01 03:34:00,1900-01-01 06:00:00,LEA,G,3,245,68,NaN,...,2001-01-01,1.0,158,12,146.0,0.0,0,2001-01-01 03:22:00,0 days,978348120.0
4,2330,03:39:00,1900-01-01 03:39:00,1900-01-01 03:39:00,PAL,G,16,245,52,NaN,...,2001-01-01,1.0,0,0,0.0,0.0,0,2001-01-01 03:39:00,0 days,978349140.0


In [101]:
# Converting pandas dataframe to csv and saving it. 
df.to_csv('wishful_thinking.csv')